In [1]:
import pandas as pd
import datetime

df = pd.read_csv("operations.csv",sep = ";",index_col=None)
df

,date,asset,qtd bought,qtd sold,unit value,value bought,value sold
0,19/02/2021,PETR3,200,0,27.19,5438.0,0.0
1,19/02/2021,PETR3,100,0,27.19,2719.0,0.0
2,19/02/2021,PETRC300,0,300,0.47,0.0,141.0
3,22/02/2021,PETRC300,300,0,0.09,27.0,0.0
4,13/01/2021,VVARN120,1000,0,0.11,110.0,0.0
5,20/01/2021,VVARB170,1000,0,0.10,100.0,0.0
6,29/01/2021,VVARB170,0,1000,0.21,0.0,210.0
7,29/01/2021,VVARN120,0,1000,0.09,0.0,90.0


In [2]:
#pattern = "( \\((MoM|QoQ)\\))"
pattern_of_options_asset = "(([^A-Z]*[A-Z]){5}([0-9]){2,3})"
filter = df['asset'].str.contains(pattern_of_options_asset)
#df_option_only = df[{"asset" : [pattern_of_options_asset]}]
df_option_only  = df[filter]
df_option_only = df_option_only.reset_index() #if not reset, get error on adding the month
df_option_only

/home/hyodo/.local/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,index,date,asset,qtd bought,qtd sold,unit value,value bought,value sold
0,2,19/02/2021,PETRC300,0,300,0.47,0.0,141.0
1,3,22/02/2021,PETRC300,300,0,0.09,27.0,0.0
2,4,13/01/2021,VVARN120,1000,0,0.11,110.0,0.0
3,5,20/01/2021,VVARB170,1000,0,0.10,100.0,0.0
4,6,29/01/2021,VVARB170,0,1000,0.21,0.0,210.0
5,7,29/01/2021,VVARN120,0,1000,0.09,0.0,90.0


In [3]:
dates = df_option_only["date"]
dates

0    19/02/2021
1    22/02/2021
2    13/01/2021
3    20/01/2021
4    29/01/2021
5    29/01/2021
Name: date, dtype: object

In [4]:
# esse eh o melhor jeito de fazer a transformação ?
data = []
for index, value in dates.items():
    date = datetime.datetime.strptime(value, "%d/%m/%Y")
    data.append(date.month)    
month_dates = pd.Series(data)
month_dates

0    2
1    2
2    1
3    1
4    1
5    1
dtype: int64

In [5]:
# como fazer sem alterar o original? queria ter df_opertions_by_month
#df["dates"] = month_dates
df_opertions_by_month = df_option_only.copy()
df_opertions_by_month['month'] = month_dates
df_opertions_by_month

,index,date,asset,qtd bought,qtd sold,unit value,value bought,value sold,month
0,2,19/02/2021,PETRC300,0,300,0.47,0.0,141.0,2
1,3,22/02/2021,PETRC300,300,0,0.09,27.0,0.0,2
2,4,13/01/2021,VVARN120,1000,0,0.11,110.0,0.0,1
3,5,20/01/2021,VVARB170,1000,0,0.10,100.0,0.0,1
4,6,29/01/2021,VVARB170,0,1000,0.21,0.0,210.0,1
5,7,29/01/2021,VVARN120,0,1000,0.09,0.0,90.0,1


In [6]:
#df_group_by_month =  df.groupby(["date","asset"]).sum()
df_group_by_month = df_opertions_by_month.groupby(["month","asset"]).sum().reset_index()
#df_group_by_month= df_group_by_month.rename(columns = {"date" : "month"})
df_group_by_month.head()

,month,asset,index,qtd bought,qtd sold,unit value,value bought,value sold
0,1,VVARB170,11,1000,1000,0.31,100.0,210.0
1,1,VVARN120,11,1000,1000,0.20,110.0,90.0
2,2,PETRC300,5,300,300,0.56,27.0,141.0


In [7]:
#df_month_with_total = pd.DataFrame([])
df_month_with_total = df_group_by_month.copy()
df_month_with_total["total"] = df_month_with_total.apply(lambda row: row["value sold"] - row["value bought"], axis=1)
df_month_with_total

,month,asset,index,qtd bought,qtd sold,unit value,value bought,value sold,total
0,1,VVARB170,11,1000,1000,0.31,100.0,210.0,110.0
1,1,VVARN120,11,1000,1000,0.20,110.0,90.0,-20.0
2,2,PETRC300,5,300,300,0.56,27.0,141.0,114.0


In [8]:
keys_for_end_of_month = ["value bought","value sold","total"]
#df_end_of_month_with_total =  df_month_with_total.groupby(["month"])[keys_for_end_of_month].sum()
df_end_of_month_with_total =  df_month_with_total.groupby(["month"])[keys_for_end_of_month].sum().reset_index()

df_end_of_month_with_total.head()

,month,value bought,value sold,total
0,1,210.0,300.0,90.0
1,2,27.0,141.0,114.0


In [10]:
df_end_of_month_with_total_and_tax = df_end_of_month_with_total.copy()
df_end_of_month_with_total_and_tax["tax"] = df_end_of_month_with_total_and_tax.apply(lambda row: row["total"] * 0.15, axis=1)
df_end_of_month_with_total_and_tax

,month,value bought,value sold,total,tax
0,1,210.0,300.0,90.0,13.5
1,2,27.0,141.0,114.0,17.1
